# Lab 9 - Detección de Anomalías con Autoencoder, Isolation Forest y LOF

In [ ]:
#!pip install -r requirements

In [1]:
from sklearn.datasets import fetch_covtype
import numpy as np 
import pandas as pd


In [17]:
bunch = fetch_covtype(as_frame=True)

X: pd.DataFrame = bunch.data 
y: pd.Series = bunch.target

df = X.copy()
df['Cover_Type'] = y
print(X.shape, y.shape)

(581012, 54) (581012,)


In [18]:
type(df)

pandas.core.frame.DataFrame

In [19]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


### Standardization and Train/Test/Val

In [24]:
y_bin = (y!=2).astype(int)
X_normal = X[y_bin == 0]
X_anomal = X[y_bin == 1]

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


test_normals = X_normal.sample(frac=0.2, random_state=32)
test_anoms   = X_anomal.sample(frac=0.2, random_state=32)

X_test = pd.concat([test_normals, test_anoms]).sample(frac=1, random_state=32)

y_test = pd.Series(0, index=X_test.index)
y_test.loc[X_test.index.intersection(X_anomal.index)] = 1  # anomalies = 1

X_normal_remaining = X_normal.drop(test_normals.index) # drop samples from remaining on testing.
X_train, X_val = train_test_split(X_normal_remaining, test_size=0.25, random_state=32) 


In [26]:



binary_cols = [c for c in X_train.columns if set(X[c].unique()).issubset({0, 1})]
cols_to_scale = [c for c in X_train.columns if c not in binary_cols]

scaler = StandardScaler().fit(X_train[cols_to_scale])

def apply_scaler(df: pd.DataFrame) -> pd.DataFrame:
    df_scaled = df.copy()
    df_scaled[cols_to_scale] = scaler.transform(df[cols_to_scale])
    return df_scaled
X_train_s = X_train.copy()
X_train_s = apply_scaler(X_train)
X_val_s   = X_val.copy()
X_val_s   = apply_scaler(X_val)
X_test_s  = X_test.copy()
X_test_s  = apply_scaler(X_test)